# EECS 545 (WN 2024) Homework 4: RNNs and Image Captioning

<span class="instruction">Before starting the assignment, please fill in the following cell.</span>

In [ ]:
###################################################################
# Enter your first and last name, e.g. "John Doe"                 #
# for example                                                     #
# __NAME__ = "Anthony Liu"                                        #
# __UNIQID__ = "anthliu"                                          #
###################################################################
raise NotImplementedError("TODO: Add your implementation here.")
###################################################################
#                        END OF YOUR CODE                         #
###################################################################

print(f"Your name and email: {__NAME__} <{__UNIQID__}@umich.edu>")
assert __NAME__ and __UNIQID__

# RNNs and Image Captioning
In this notebook, you will test your RNN implementation from `rnn.py` on the coco image captioning dataset.

## Setup code
Before getting started, we need to run some boilerplate code to set up our environment. You'll need to rerun this setup code each time you start the notebook. Let's start by checking whether we are using Python 3.11 or higher.

In [ ]:
import sys
if sys.version_info[0] < 3:
    raise Exception("You must use Python 3")

if sys.version_info[1] < 11:
    print("Autograder will execute your code based on Python 3.11 environment. Please use Python 3.11 or higher to prevent any issues")
    print("You can create a conda environment with Python 3.11 like 'conda create --name eecs545 python=3.11'")
    raise Exception("Python 3 version is too low: {}".format(sys.version))
else:
    print("You are good to go")

First, run this cell load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html) extension. This allows us to edit `.py` source files, and re-import them into the notebook for a seamless editing and debugging experience.

In [ ]:
%load_ext autoreload
%autoreload 2

Then, we run some setup code for this notebook: Import some useful packages and increase the default figure size.

In [ ]:
# install required libraries
# !pip install numpy==1.24.1 matplotlib==3.6.2 scikit-learn==1.2.0 h5py==3.8.0 imageio==2.25.1

# import libraries
import math
import numpy as np
import matplotlib.pyplot as plt

# set figure size
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 16

In [ ]:
from IPython.display import display_html, HTML

display_html(HTML('''
<style type="text/css">
  .instruction { background-color: yellow; font-weight:bold; padding: 3px; }
</style>
'''));

Running the following cell allow us to import from `rnn_layers.py` and `rnn.py`. If it works correctly, it should print the message:
```Hello from rnn_layers.py``` and ```Hello from rnn.py```

In [ ]:
from rnn_layers import hello
from rnn import hello as hello2
hello()
hello2()

# Image Captioning


## What is COCO?

[COCO](https://cocodataset.org/) (Common Objects in COntext) is a large-scale object detection, segmentation, and captioning dataset.

COCO has 330K images (>200K labeled)! Labelled images have object segmentations, and captions.

![COCO examples](https://cocodataset.org/images/coco-examples.jpg)


## Loading COCO

Please download the dataset (987Mb) from [this link](https://drive.google.com/file/d/1RjZTIVp4ES1Ewv1QAJgCS_7DYyJa-Yf1/view?usp=sharing) and unzip the folder into the data directory.
```
HW4 /
--| image_captioning.ipynb
--| data /
----| coco_captioning /
------| coco2014_captions.h5
------| ...
```

The dataset contains the preprocessed features and captions from the COCO dataset.

In [ ]:
!pip install h5py
import coco_utils

MAX_TRAIN = 200# We'll train with a very small dataset

# Load COCO data from disk; this returns a dictionary
small_data = coco_utils.load_coco_data(max_train=MAX_TRAIN)

## Visualizing COCO

Let's check out some images in our COCO dataset. You can browse the full dataset [here](https://cocodataset.org/#explore)! My favourite image is [this one](https://cocodataset.org/#explore?id=314526). It turns out when you have a 330K image dataset, some of the images might be really [weird](https://blog.roboflow.com/coco-dataset-image-search/). Check it out!

Let's visualize some images from the downloaded COCO. Note we have to load them from the URL since they aren't stored in our dataset. (We save the preprocessed features). Some of these images may not load properly because the URLs are no longer active.

In [ ]:
def md_table(urls, captions, cols=5):
    table = '|   ' * cols + '|\n' + '|---' * cols + '|\n'
    row = lambda idxs: '| ' + ' | '.join(f'![train image {i}]({urls[i]})' for i in idxs) + ' |'
    row_cap = lambda idxs: '| ' + ' | '.join(f'**{str(captions[i])}**' for i in idxs) + ' |'
    table += '\n'.join(row(range(i, i+cols)) + '\n' + row_cap(range(i, i+cols)) for i in range(0, len(urls), cols))
    table = table.replace('<', '\<')# show angle brackets in MD
    table = table.replace('\<br>', '<br>')# keep <br> as newline
    return table

In [ ]:
from IPython.display import Markdown as md
n_imgs = 20
rng = np.random.default_rng(545)
captions, features, img_urls = coco_utils.sample_coco_minibatch(
    small_data, split='train', batch_size=n_imgs, seed=545)
cap_str = [coco_utils.decode_captions(c, small_data['idx_to_word']) for c in captions]
md(md_table(img_urls, cap_str))

Footnote: Special tokens. \<START> and \<END> tokens denote the start and end of the captions.
The \<UNK> token is created during preprocessing and replaces words that do not occur in the dataset very often. Doing this sometimes helps the model learn better since it shrinks the vocabulary!

## Training
Next, we'll train an RNN to predict captions on COCO. <span class="instruction">Make sure you have implemented `temporal_fc_forward` and `temporal_fc_backward` in `rnn_layers.py`, and `CaptioningRNN` from `rnn.py`.</span>
Read through `captioning_solver.py` to make sure you understand the API. 

### Correctness Checks

Like `cnn.ipynb`, we have included some tests to check your implementation. However, we have left the numerical gradient checks blank.
These checks are optional, but they may be helpful for debugging. Hint: follow the same steps as done in `cnn.ipynb`.

In [ ]:
from gradient_check import rel_error, eval_numerical_gradient, eval_numerical_gradient_array
from rnn_layers import temporal_fc_backward, temporal_fc_forward

# OPTIONAL TODO Test with eval_numerical_gradient_array.
# Hint: Start by sampling some random test data. (Refer to the rnn_layers.py comments to check the shapes)
# Then, call eval_numerical_gradient_array on the test data. (Refer to cnn.ipynb for examples)
# Compare the results of eval_numerical_gradient_array with your own backward implementation.


In [ ]:
from gradient_check import rel_error, eval_numerical_gradient, eval_numerical_gradient_array
from rnn import CaptioningRNN

# Generate dummy test data
wtoi = {f'{i}': i for i in range(7)}# dummy word to index
wtoi['<NULL>'] = 8
wtoi['<START>'] = 9
wtoi['<END>'] = 10
np.random.seed(0)
test_rnn_model = CaptioningRNN(
      cell_type='rnn',
      word_to_idx=wtoi,
      input_dim=5,
      hidden_dim=4,
      wordvec_dim=5,
)
rng = np.random.default_rng(545)
X = rng.standard_normal((2, 5))
y = rng.integers(0, 10, size=(2, 4))

print('Testing training loss...')
loss, grads = test_rnn_model.loss(X, y)

assert np.abs(loss - 7.783337014604364) < 1e-4, 'loss should be close.'

# OPTIONAL TODO Test with eval_numerical_gradient_array.
# Hint: write a loop to check each grad (Refer to cnn.ipynb for examples)


print('done.')

In [ ]:
# Generate dummy test data
wtoi = {f'{i}': i for i in range(7)}# dummy word to index
wtoi['<NULL>'] = 8
wtoi['<START>'] = 9
wtoi['<END>'] = 10
np.random.seed(1)
test_rnn_model = CaptioningRNN(
      cell_type='rnn',
      word_to_idx=wtoi,
      input_dim=5,
      hidden_dim=4,
      wordvec_dim=5,
)
rng = np.random.default_rng(545)
X = rng.standard_normal((2, 5))

print('Testing rnn sample...')
captions = test_rnn_model.sample(X, max_length=17)
gt_captions = np.array([[1, 0, 7, 2, 1, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
       [6, 5, 1, 0, 7, 6, 5, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7]])
assert np.all(captions == gt_captions)
print('Done.')

### Initializing the model

In [ ]:
from rnn import CaptioningRNN
from captioning_solver import CaptioningSolver

np.random.seed(0)
# Experiment with vanilla RNN
small_rnn_model = CaptioningRNN(
      cell_type='rnn',
      word_to_idx=small_data['word_to_idx'],
      input_dim=small_data['train_features'].shape[1],
      hidden_dim=512,
      wordvec_dim=256,
)

small_rnn_solver = CaptioningSolver(small_rnn_model, small_data,
       update_rule='adam',
       num_epochs=58,
       batch_size=25,
       optim_config={
         'learning_rate': 4e-3,
       },
       lr_decay=0.95,
       verbose=True, print_every=10,
     )

### Train!

Remember to include the plot of your learning curves in the final report.

In [ ]:
small_rnn_solver.train()

In [ ]:
# Plot the training losses
plt.plot(small_rnn_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.savefig('image_captioning_loss.png', dpi=256)
plt.show()
plt.close()

### Testing and Visualization

Finally, let's test our model on some validation data. Include your generated train and validation captions in the final report.

In [ ]:
from image_utils import image_from_url

tables = {}
for split in ['train', 'val']:
    # some images might be deprecated. You may rerun the code several times
    # to successfully get the sample images from url.
    minibatch = coco_utils.sample_coco_minibatch(
        small_data, split=split, batch_size=10, seed=545)
    gt_captions, features, urls = minibatch
    gt_captions = coco_utils.decode_captions(gt_captions,
                                             small_data['idx_to_word'])

    sample_captions = small_rnn_model.sample(features)
    sample_captions = coco_utils.decode_captions(sample_captions,
                                                 small_data['idx_to_word'])

    figure_caption = [f'Sample: {sample_cap} <br> GT: {gt_cap}' for sample_cap, gt_cap in zip(sample_captions, gt_captions)]
    tables[split] = md_table(urls, figure_caption)


In [ ]:
md('## Training samples \n ' + tables['train'])

In [ ]:
md('## Validation samples \n ' + tables['val'])